# AutoGrader for Jupyter/Python scripts submitted to Blackboard
This is a program that allows you to quickly grade student code by making simple annotations to your solution files. It compares the outputs of your solutions to the outputs of the student solutions when given the same set of parameters.

## Installation
Before proceeding make sure you install the required packages using REQUIREMENTS.txt

## Limitations
* Requires a Unix machine to run auto-grading as the timeout functionality uses Unix Signals
* You cannot use underscores in the titles of the Blackboard assignments
* Cannot grade class initializers (\_\_init\_\_ functions)
* Students cannot use global variables as code outside functions and classes is deleted by code_parser.py
* Can only grade top-level functions and top-level classes, no classes within classes or anything like that
* Does not currently support comparing of print() statements (It could be done, but it's just easier if you make the students return a string)
* All problems are graded from 0 to 1 and weighted equally in the final score (not counting extra credit). If you want to change the scalings you can do so with the output Excel spreadsheet.

## TODO
* Allow each problem to pass timeout_s instead of defining a single one. Currently, it is a constant in student_code.py

## Before Starting
* Please add \_\_str()\_\_ and \_\_repr()\_\_ functions to your classes so the student gets more detailed feedback on their failed test cases

# How It Works (Design and Implementation)
The basic principle is that we want to compare the output of our solution to the output of the student's solution when given the same set of parameters. If they differ, or an exception occurs, or the student code takes too long, we count it as a failed test case and give as much detailed feedback to the student as possible about why their code didn't pass the test case. We tell the AutoGrader how to generate sets of parameters through function annotations, which allows us to quickly grade any assignment.

The general pipeline is as follows:
1. Prepare grader by parsing command-line arguments and creating directories to be used later
2. Override some libraries we don't want to run while grading such as matplotlib's show() or Colab's file.upload()
3. Import the solution file functions and classes while parsing the annotations
4. Convert jupyter .ipynb notebooks to .py files
5. Grade all student code files

# How To Run
Let's say we are grading an exercise named "Example Exercise 1" on Blackboard for which we have a solution file called **example_exercise1_solution.py**. You would then follow these steps.

1. Download all the submissions as a single zip file from Blackboard following these instructions:
> * Go to the Grade Center
> * Click the arrow next to the assignment column
> * Select **File Assignment Download**
> * Select all students and all attempts
> * Click **Download** and then download the file

2. Rename the zip file to **example_exercise1.zip** and move it to the "**\_data\_**" folder. **YOU DON'T NEED TO EXTRACT** as the grader will do it automatically and create a folder named **example_exercise1** with all the student submissions inside. If you do extract them, pass the directory as command-line argument **--student_dir**

3. Move the solution file to the "**\_solution\_**" folder. You can place it anywhere in your computer, just make sure to pass the correct path to command-line argument **solution_file**

4. Your directory structure should then look like this:
```
github-clone/_data_/example_exercise1.zip <---- This is the blackboard file
github-clone/_solutions_/example_exercise1_solution.py <----- Annotated solution
```

5. Annotate the solution file (details in next section)

6. Run **run_grader.py** with the correct command-line parameters

The command-line arguments of **run_grader.py** are:
* --solution_file (path to solution file)
* --student_dir (path to folder where zip file is)
* --max_grade (float)
* --multiprocessing (number of CPU cores to use, optional)
* --students (utep usernames of specific students to grade, separated by spaces, optional)

And they also have short-hand names being:
* -sol (...)
* -sd (...)
* -mg (...)
* -mp (...)
* -s (...)

For example, grade the example assignment with a max grade of 100, grading 15 students in parallel with 15 cores
> python run_grader.py -sol \_solutions\_/example\_exercise1\_solution.py -sd \_data\_/example\_exercise1 -mg 100 -mp 15

If **student_dir** has the same name as the solution (minus the "_solution" part) and is located in the "**\_data\_**" folder as in this example you don't need to pass it as an argument. The program will find it automatically. Thus, we can do
> python run_grader.py -sol \_solutions\_/example\_exercise1\_solution.py -mg 100 -mp 15

To grade individual students "jperez", "ofuentes", "aarnal" we can do
> python run_grader.py -sol \_solutions\_/example\_exercise1\_solution.py -mg 100 -mp 15 **-s jperez ofuentes aarnal**

# How To Annotate (Top-Level Functions)
Let's say the solution function is

In [ ]:
def list_has_k(L, k):
    return k in L

We want to generate random lists **L** and random numbers **k**. We can create a function that tells the grader how to generate each parameter as follows:

In [ ]:
import numpy as np
def generate_list():
    return list(np.random.random_integers(0, 100), 100)

def generate_k():
    return np.random.randint(-100, 200)

Then we annotate the function with **@grader.generate_test_case(...)** and pass the functions with named parameters matching those in the graded function.

We can also annotate functions with **@grader.no_test_cases()** when we want a function to not be graded.

In [ ]:
import numpy as np

import grader

@grader.generate_test_case(L = generate_list, k = generate_k)
def list_has_k(L, k):
    return k in L

@grader.no_test_cases()
def skip_grading():
    return True

Note that @grader.generate_test_case() takes a **\_\_trials\_\_** parameter that defines how many test cases will be generated and which defaults to 2500 but can be easily changed.

Also note that since Python supports lambda functions we can do something like this when the parameter generation functions are short:

In [ ]:
import numpy as np

import grader

@grader.generate_test_case(__trials__ = 500,
                           L = lambda: list(np.random.random_integers(0, 100), 100), 
                           k = lambda: np.random.randint(-100, 200))
def list_has_k(L, k):
    return k in L


By default the AutoGrader compares the student output and the solution output to see if the student got the right answer. 
> You can check the details of this in the file **grader.py** through the function **compare_outputs()**.

If you wanted more control, you can use a custom comparer annotation that gives you both the solution (sol) and student (stu) outputs and the parameters.

In the following function we convert the outputs to sets to check if they are the same as they return lists in the original problem.

In [ ]:
def compare_function(solution_output, student_output, solution_parameters, student_parameters):
    return set(solution_output) == set(student_output)

#@grader.generate_test_case(S = ..., words = ...)
@grader.generate_custom_comparer(compare_function)
def words_in_largest_set(S, words):
    ls = [len(s) for s in S]
    ls = S[np.argmax(ls)]
    wls = [words[i] for i in ls]
    return wls

Using lambda notation:

In [ ]:
#@grader.generate_test_case(S = ..., words = ...)
@grader.generate_custom_comparer(lambda sol_out, stu_out, sol_params, stu_params: set(sol_out) == set(stu_out))
def words_in_largest_set(S, words):
    ls = [len(s) for s in S]
    ls = S[np.argmax(ls)]
    wls = [words[i] for i in ls]
    return wls

# How to Annotate (Class Functions)
Let's say we have the following class and class functions which we want to grade.

* The first function **lowercase_name()** takes no parameters and returns a string.

* The second function **get_score_k(k)** takes 1 parameter and returns either None or an int.

* The third function **compute_lab_average()** has no parameters, but doesn't return and instead sets a value inside the class.

* The fourth function **print_info()** is a utility function and will not be graded.

In [ ]:
class Student:
    def __init__(self, name = 'Jose', lab_scores = [100, 100, 100]):
        self.name = name
        self.lab_scores = lab_scores

        self.average_lab = -1
    
    def lowercase_name(self):
        return self.name.lower()
    
    def get_score_k(self, k):
        if k < 0 or k >= len(self.lab_scores):
            return None
        return self.lab_scores[k]
    
    def compute_lab_average(self):
        self.average_lab = sum(self.lab_scores) / len(self.lab_scores)

    def print_info(self):
        print('Name=', self.name, 'Scores=', self.lab_scores, 'Average=', self.average_lab)

We **import grader** and start by annotating the functions we don't want to be graded with **@grader.no_test_cases()**. Although **\_\_init\_\_** is ignored by default, we will add an annotation for clarity sake anyway.

In [ ]:
import grader
class Student:
    @grader.no_test_cases()
    def __init__(self, name = 'Jose', lab_scores = [100, 100, 100]):
        self.name = name
        self.lab_scores = lab_scores

        self.average_lab = -1

    @grader.no_test_cases()
    def print_info(self):
        print('Name=', self.name, 'Scores=', self.lab_scores, 'Average=', self.average_lab)

For the first function we need to tell the grader how to generate class instances. We do this by passing functions with named parameters matching those in the construction and using the **@grader.generate_class()** annotation.

**DO NOT FORGET** to also add **@grader.generate_test_case()**. This function has no parameters so we can pass the annotation without any parameters as well.

In [ ]:
import grader


def generate_name():
    names = ['Jose', 'Olac', 'Diego']
    return str(np.random.choice(names, 1)[0])


def generate_lab_scores():
    return np.random.random_integers(0, 100, 10)


class Student:
    @grader.no_test_cases()
    def __init__(self, name='Jose', lab_scores=[100, 100, 100]):
        self.name = name
        self.lab_scores = lab_scores

        self.average_lab = -1

    @grader.generate_class(__trials_per_instance__ = 5,
                           name = generate_name,
                           lab_scores = generate_lab_scores)
    @grader.generate_test_case()
    def lowercase_name(self):
        return self.name.lower()


Note that **@grader.generate_class()** takes a **\_\_trials\_per_instance\_\_** parameter which has **NO DEFAULT VALUE** and is required to be given. This parameter specifies how many test cases will be generated before a new class instance should be created.

We can also use lambda notation as before to shorten this to:

In [ ]:
import grader


class Student:
    @grader.no_test_cases()
    def __init__(self, name='Jose', lab_scores=[100, 100, 100]):
        self.name = name
        self.lab_scores = lab_scores

        self.average_lab = -1

    @grader.generate_class(__trials_per_instance__=5,
                           name=lambda: str(np.random.choice(['Jose', 'Olac', 'Diego'], 1)[0]),
                           lab_scores=lambda: np.random.random_integers(0, 100, 10))
    @grader.generate_test_case()
    def lowercase_name(self):
        return self.name.lower()


Using what we know so far we can easily annotate the second problem

In [ ]:
import grader

class Student:
    @grader.no_test_cases()
    def __init__(self, name='Jose', lab_scores=[100, 100, 100]):
        self.name = name
        self.lab_scores = lab_scores

        self.average_lab = -1

    @grader.generate_class(__trials_per_instance__=5,
                           name=lambda: str(np.random.choice(['Jose', 'Olac', 'Diego'], 1)[0]),
                           lab_scores=lambda: np.random.random_integers(0, 100, 10))
    @grader.generate_test_case(k = lambda: np.random.randint(-5, 100))
    def get_score_k(self, k):
        if k < 0 or k >= len(self.lab_scores):
            return None
        return self.lab_scores[k]

For the third problem, we will use what we know so far and a custom comparer to compare the class variable "self.average_lab" as the function does not return anything but instead updates an internal class variable.

To help us we will use **grader.compare_outputs()** which returns True when the two given parameters are equal to each other and is the utility function used internally for grading all other problems.

In [ ]:
import grader

def compare_function(solution_instance, student_instance, solution_output, student_output, solution_parameters, student_parameters):
    return grader.compare_outputs(solution_instance.average_lab, student_instance.average_lab)

class Student:
    @grader.no_test_cases()
    def __init__(self, name='Jose', lab_scores=[100, 100, 100]):
        self.name = name
        self.lab_scores = lab_scores

        self.average_lab = -1

    @grader.generate_class(__trials_per_instance__=5,
                           name=lambda: str(np.random.choice(['Jose', 'Olac', 'Diego'], 1)[0]),
                           lab_scores=lambda: np.random.random_integers(0, 100, 10))
    @grader.generate_test_case()
    @grader.generate_custom_comparer(compare_function)
    def compute_lab_average(self):
        self.average_lab = sum(self.lab_scores) / len(self.lab_scores)

Using lambda notation

In [ ]:
import grader

class Student:
    @grader.no_test_cases()
    def __init__(self, name='Jose', lab_scores=[100, 100, 100]):
        self.name = name
        self.lab_scores = lab_scores

        self.average_lab = -1

    @grader.generate_class(__trials_per_instance__=5,
                           name=lambda: str(np.random.choice(['Jose', 'Olac', 'Diego'], 1)[0]),
                           lab_scores=lambda: np.random.random_integers(0, 100, 10))
    @grader.generate_test_case()
    @grader.generate_custom_comparer(lambda sol_instnc, stu_instnc, sol_output, stu_output, sol_params, stu_params: grader.compare_outputs(sol_instnc.average_lab, stu_instnc.average_lab))
    def compute_lab_average(self):
        self.average_lab = sum(self.lab_scores) / len(self.lab_scores)